# Using structured JSON output feature from OpenAI
Sources: 
- OpenAI web page for JSON formatting rules to use 'strict' mode: https://platform.openai.com/docs/guides/structured-outputs
- A Python Package for high-level usage of LLM structured output: https://github.com/dottxt-ai/outlines

In [102]:
# !pip uninstall -y sympy pysb torch outlines
# !pip install pysb sympy==1.11.1
# !pip install torch sympy==1.13.1 --no-deps
# !pip install outlines --no-deps

In [1]:
import sys
from pathlib import Path
import os
# get current path
sys.path.append(str(Path.cwd().parent))

## Providing the JSON schema directly as the response format
- I.e. Provide the indra JSON schema to the `response_format` parameter of `OpenAI.beta.chat.completions.parse` function of OpenAI

OpenAI enforces a subset of the 'JSON Schema' language. The below is a list of non-exhaustive rules that is enforced:
1. root JSON object is 'object' type
2. 'required' includes all and only the fields in 'property' 
    - This ensures LLM doesn't generate any key-val pair in 'property' that is not defined in the schema.
    - To make a defined field in 'property' optional, we can add "null" as an optional type of that field. 
3. 'additionalProperties' field is set to false and is included for every 'object' type.
4. Each sub-schema in 'anyOf', namely "#/definitions/RegulateActivity", "#/definitions/Modification", "#/definitions/SelfModification", etc. has a different first key
   in their 'property' field. <br>
   This is achievable by adding a new field with a unique constant value. E.g. "#/definitions/RegulateActivity" has 
   ```
   "kind": {
                "type": "string",
                "const": "RegulateActivity"
            }
    ```
    And "#/definitions/Modification" has
    ```
    "kind": {
                "type": "string",
                "const": "Modification"
            }
    ```

- The below schema follows all the rules for OpenAI 'strict' mode, except for one. It contains the 'allOf' syntax, which OpenAI doesn't currently support.
- We can resolve 'allOf' syntax by merging the items and removing the 'allOf' syntax.

In [2]:
import json
import copy
from indra.statements.io import stmts_from_json
import pandas as pd

In [3]:
# Load the JSON schema
schema_path = "/Users/thomaslim/gyorilab/indra_gpt/indra_gpt/resources/indra_schema_openai_v3.json"
with open(schema_path, "r") as f:
    schema = json.load(f)

In [4]:
def resolve_ref(schema, ref_path):
    """Helper function to resolve a $ref path in a JSON schema."""
    keys = ref_path.lstrip("#/").split("/")
    ref_obj = schema
    for key in keys:
        ref_obj = ref_obj.get(key, {})
    return copy.deepcopy(ref_obj)  # Return a deep copy to prevent modifying the original schema

def merge_allOf(schema, root_schema):
    """Recursively merges allOf definitions into their parent objects and removes allOf.
       - Resolves $ref only if inside allOf
       - Resolves nested allOf in properties, items, and definitions
       - Does NOT resolve nested $ref inside resolved object
    """
    if isinstance(schema, dict):
        if "allOf" in schema:
            merged_schema = {}
            required_fields = set()

            for sub_schema in schema["allOf"]:
                if "$ref" in sub_schema:
                    ref_obj = resolve_ref(root_schema, sub_schema["$ref"])
                    sub_schema = ref_obj.copy()  # Use a copy to prevent modifying the root schema

                # Merge properties correctly
                for key, value in sub_schema.items():
                    if key == "required":
                        required_fields.update(value)
                    elif key in merged_schema and isinstance(merged_schema[key], dict) and isinstance(value, dict):
                        merged_schema[key].update(value)  # Merge nested dictionaries (e.g., properties)
                    else:
                        merged_schema[key] = value  # Overwrite other keys

            merged_schema.pop("allOf", None)  # Remove allOf after merging
            if required_fields:
                merged_schema["required"] = list(required_fields)  # Assign merged required fields
            
            schema.clear()
            schema.update(merged_schema)

        # Recursively process properties, items, and definitions **after merging**
        for key in ["properties", "items", "definitions"]:
            if key in schema and isinstance(schema[key], dict):
                schema[key] = {k: merge_allOf(v, root_schema) for k, v in schema[key].items()}

        return schema

    elif isinstance(schema, list):
        return [merge_allOf(item, root_schema) for item in schema]

    return schema  # Return primitive values unchanged

In [5]:
# Resolve `allOf` occurrences while resolving only first-level refs inside 'allOf'
post_processed_schema = merge_allOf(schema, schema)

In [6]:
post_processed_schema

{'type': 'object',
 'properties': {'statements': {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/RegulateActivity'},
     {'$ref': '#/definitions/Modification'},
     {'$ref': '#/definitions/SelfModification'},
     {'$ref': '#/definitions/ActiveForm'},
     {'$ref': '#/definitions/Gef'},
     {'$ref': '#/definitions/Gap'},
     {'$ref': '#/definitions/Complex'},
     {'$ref': '#/definitions/Association'},
     {'$ref': '#/definitions/Translocation'},
     {'$ref': '#/definitions/RegulateAmount'},
     {'$ref': '#/definitions/Conversion'}]}}},
 'required': ['statements'],
 'additionalProperties': False,
 'definitions': {'RegulateActivity': {'type': 'object',
   'properties': {'kind': {'type': 'string', 'const': 'RegulateActivity'},
    'type': {'type': 'string',
     'enum': ['Activation', 'Inhibition'],
     'description': 'The type of the statement'},
    'subj': {'$ref': '#/definitions/Agent'},
    'obj': {'$ref': '#/definitions/Agent'},
    'obj_activity': {'type':

### Use `outlines` package to use OpenAI's 'structured outputs' feature.

In [7]:
import outlines

# Use OpenAI API instead of local model
model = outlines.models.openai("gpt-4o")  # Or "gpt-4-turbo" for cheaper API cost
# Define generators
generator = outlines.generate.json(model, json.dumps(post_processed_schema))
# Sample Prompt
prompt = "We found that 24 hours of BMP pretreatment caused a doubling in PTEN half-life (15.1 hours to 28.4 hours, p = 0.03, n = 4)."

In [8]:
result_v1 = generator(prompt)

INFO: [2025-02-10 10:20:59] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [9]:
print(json.dumps(result_v1, indent=4))

{
    "statements": [
        {
            "kind": "RegulateAmount",
            "type": "IncreaseAmount",
            "subj": {
                "name": "BMP",
                "mods": [],
                "mutations": [],
                "bound_conditions": [],
                "activity": {
                    "activity_type": "regulation",
                    "is_active": true
                },
                "location": "",
                "db_refs": {},
                "sbo": ""
            },
            "obj": {
                "name": "PTEN",
                "mods": [],
                "mutations": [],
                "bound_conditions": [],
                "activity": {
                    "activity_type": "stability",
                    "is_active": true
                },
                "location": "",
                "db_refs": {},
                "sbo": ""
            },
            "evidence": [
                {
                    "text": "24 hours of BMP pretreatment

## Apply to benchmark corpus

In [20]:
indra_benchmark_corpus_sample_50 = json.load(open("/Users/thomaslim/gyorilab/indra_gpt/indra_gpt/resources/indra_benchmark_corpus_sample_50.json", "r"))

In [21]:
# Assuming indra_benchmark_corpus_all_correct is a list of JSON objects
data = [
    {
        'text': obj['evidence'][0]['text'] if 'evidence' in obj and obj['evidence'] else None,
        'original_json_statement': obj
    }
    for obj in indra_benchmark_corpus_sample_50
]

# Convert list to DataFrame
df = pd.DataFrame(data)

In [23]:
# Now ally the indra.statements.io.stmt_from_json to convert the extracted_statement_json to a list of INDRA statements
for i, row in df.iterrows():
    try:
        indra_statement_from_original = stmts_from_json([row.original_json_statement], on_missing_support='handle')
        df.at[i, 'indra_statements_from_original'] = indra_statement_from_original
    except Exception as e:
        df.at[i, 'indra_statements_from_original'] = str(e)

In [26]:
# Use generator_v1 and apply to text column, and add a new column called extracted_statement_json
for i, row in df.iterrows():
    try:
        extracted_statement_json = generator(row.text)
        df.at[i, 'extracted_statement_json'] = extracted_statement_json
    except Exception as e:
        df.at[i, 'extracted_statement_json'] = str(e)

INFO: [2025-02-10 10:24:09] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-10 10:24:09] openai._base_client - Retrying request to /chat/completions in 0.381706 seconds
INFO: [2025-02-10 10:24:34] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-10 10:24:34] openai._base_client - Retrying request to /chat/completions in 0.385108 seconds
INFO: [2025-02-10 10:24:54] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-10 10:24:54] openai._base_client - Retrying request to /chat/completions in 0.444334 seconds
INFO: [2025-02-10 10:25:25] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-10 10:25:25] openai._base_client - Retrying request to /chat/completions in 0.448398 seconds
INFO: [2025-02-10 10:25:40] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/

In [27]:
# Recursively go through each key-value, and if it is an empty string or list or dict, remove the key-value pair
def remove_empty_strings_and_lists(d):
    for key, value in list(d.items()):
        if isinstance(value, dict):
            remove_empty_strings_and_lists(value)
        elif isinstance(value, list):
            for i in value:
                if isinstance(i, dict):
                    remove_empty_strings_and_lists(i)
        if value in ["", [], {}]:
            del d[key]
    return d

In [49]:
# Now ally the indra.statements.io.stmt_from_json to convert the extracted_statement_json to a list of INDRA statements
for i, row in df.iterrows():
    try:
        cleaned = remove_empty_strings_and_lists(row.extracted_statement_json['statements'])
        indra_statement_from_generated = stmts_from_json(cleaned['statements'], on_missing_support='handle')
        df.at[i, 'indra_statement_from_generated'] = indra_statement_from_generated
    except Exception as e:
        df.at[i, 'indra_statement_from_generated'] = str(e)

In [50]:
cleaned

{'statements': [{'kind': 'Modification',
   'type': 'Ubiquitination',
   'enz': {'name': 'cIAP1',
    'activity': {'activity_type': 'activity', 'is_active': True}},
   'sub': {'name': 'IKKgamma',
    'activity': {'activity_type': 'activity', 'is_active': True}},
   'residue': 'K',
   'position': '285',
   'evidence': [{'text': 'Moreover, we show that mutation of SUMO acceptor sites affects binding of TRAF6 and cIAP1 (D), which interacts inducibly with IKKgamma (H) and ubiquitinates IKKgamma at K285 in cotransfection experiments (D).',
     'source_api': 'Custom'}],
   'id': '1'},
  {'kind': 'Complex',
   'type': 'Complex',
   'members': [{'name': 'TRAF6',
     'activity': {'activity_type': 'activity', 'is_active': True}},
    {'name': 'cIAP1',
     'activity': {'activity_type': 'activity', 'is_active': True}}],
   'evidence': [{'text': 'Moreover, we show that mutation of SUMO acceptor sites affects binding of TRAF6 and cIAP1 (D), which interacts inducibly with IKKgamma (H) and ubiquiti

In [47]:
df

,text,original_json_statement,indra_statements_from_original,extracted_statement_json,indra_statement_from_generated
0,We found that 24 hours of BMP pretreatment cau...,"{'type': 'Activation', 'subj': {'name': 'BMP',...","[Activation(BMP(), PTEN())]",Incompatible indexer with Series,"string indices must be integers, not 'str'"
1,"In the present study, we have established that...","{'type': 'Activation', 'subj': {'name': 'CXCL1...","[Activation(CXCL12(), AKT())]","{'statements': [{'kind': 'RegulateActivity', '...",'list' object has no attribute 'items'
2,These HY specific CD8+ T cells produced interf...,"{'type': 'Activation', 'subj': {'name': 'KDM5D...","[Activation(KDM5D(), IFNG())]","{'statements': [{'kind': 'ActiveForm', 'type':...",'list' object has no attribute 'items'
3,"In support of this idea, the ability of the FH...","{'type': 'Activation', 'subj': {'name': 'FHOD1...","[Activation(FHOD1(), TAGLN())]","{'statements': [{'kind': 'RegulateActivity', '...",'list' object has no attribute 'items'
4,These observations demonstrate that STING indu...,"{'type': 'Activation', 'subj': {'name': 'DDX58...","[Activation(DDX58(), STING1())]","{'statements': [{'kind': 'ActiveForm', 'type':...",'list' object has no attribute 'items'
5,"To validate the Affymetrix microarray data, we...","{'type': 'Activation', 'subj': {'name': 'KDM3A...","[Activation(KDM3A(), MIGA1())]","{'statements': [{'kind': 'RegulateAmount', 'ty...",'list' object has no attribute 'items'
6,We found that clinically relevant levels of Hc...,"{'type': 'Activation', 'subj': {'name': 'AHCY'...","[Activation(AHCY(), DNMT3B())]","{'statements': [{'kind': 'RegulateAmount', 'ty...",'list' object has no attribute 'items'
7,The largest subunit of the human transcription...,"{'type': 'Autophosphorylation', 'enz': {'name'...","[Autophosphorylation(GTF2F(), S)]","{'statements': [{'kind': 'SelfModification', '...",'list' object has no attribute 'items'
8,This may be explained by previous studies show...,"{'type': 'Autophosphorylation', 'enz': {'name'...","[Autophosphorylation(AURKA(), S, 51)]","{'statements': [{'kind': 'SelfModification', '...",'list' object has no attribute 'items'
9,"Following ligand binding, KGFR is rapidly auto...","{'type': 'Autophosphorylation', 'enz': {'name'...","[Autophosphorylation(FGFR2(), Y)]","{'statements': [{'kind': 'SelfModification', '...",'list' object has no attribute 'items'
